In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_csv("../input/shopee-product-matching/train.csv")
test_df = pd.read_csv("../input/shopee-product-matching/test.csv")

if len(test_df) > 3:
    HAVE_TEST = True
    working_df = test_df
    del train_df
else:
    HAVE_TEST = False
    working_df = train_df
    del test_df

In [ ]:
working_df.columns

In [ ]:
if HAVE_TEST:
    working_df["matches"] = working_df["posting_id"]
    working_df[["posting_id", "matches"]].to_csv("submission.csv", index=False)
    
else:
    # target
    temp = working_df.groupby('label_group').posting_id.agg('unique').to_dict()
    working_df['target'] = working_df.label_group.map(temp)
    
    # posting_id_match
    temp = working_df.groupby('posting_id').posting_id.agg('unique').to_dict()
    working_df['posting_id_matches'] = working_df.posting_id.map(temp)

    # image match
    temp = working_df.groupby('image').posting_id.agg('unique').to_dict()
    working_df['image_matches'] = working_df.image.map(temp)
    
    # phash match
    temp = working_df.groupby('image_phash').posting_id.agg('unique').to_dict()
    working_df['phash_matches'] = working_df.image_phash.map(temp)
    
    def calculate_f1(df, target_col, pred_col):
        def f1score(row):
            n = len(np.intersect1d(row[target_col],row[pred_col]) )
            return 2*n / (len(row[target_col])+len(row[pred_col]))

        return df.apply(f1score, axis=1)
    
    posting_id_f1 = calculate_f1(working_df, "target", "posting_id_matches")
    image_f1 = calculate_f1(working_df, "target", "image_matches")
    phash_f1 = calculate_f1(working_df, "target", "phash_matches")
                                                             
    working_df[["posting_id", "posting_id_matches"]].to_csv("submission.csv", index=False)
    
    print(f"posting_id_f1: {posting_id_f1.mean(): .4f}")
    print(f"image_f1: {image_f1.mean(): .4f}")
    print(f"phash_f1: {phash_f1.mean(): .4f}")                                                             